MODELO DE CRÉDITO PARA PRÉSTAMOS PERSONALES

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier, XGBRegressor

In [30]:
"""
Función para cargar la información para entrenar y validar el modelo. 
La información no será proporcionada por el usuario final, sino que ya estará establecida. Se utilizará el archivo 
con los datos limpios y ordenados que se realizó para el modelo tradicional. 
"""
def get_data(data):
    return pd.read_csv(data)

In [31]:
"""
Función para separar la data en train y test. 
Se utilizará el 70% de los datos totales para entrenar el modelo y el 30% restante para verificar su funcionamiento. 
La división de los datos se realizará de manera aleatoria, establecionedo una semilla para poder replicar los 
resultados obtenidos. 

EXPLICAR TARGET COLUMN
"""
def split_train_test(data, target_column, random_state=100):
    data_train, data_test = train_test_split(
        data,
        test_size=0.3,
        random_state=random_state,
        stratify=data[target_column]  
    )
    return data_train.reset_index(drop=True), data_test.reset_index(drop=True)

In [50]:
"""
Función para preparar los datos. 
Ya que en los modelos que se utilizarán más adelante los datos deben de estar separados en X y Y, además de estar 
normalizados (unicamente los valores numéricos), se optó por crear una función que realizara esta preparación. Para utilizarla, es necesario alimentar a esta
función con los datos seleccionados para train y para test obtenidos en la función anterior. 
"""

def prepare_variables(data_train, data_test):
    X_train = data_train.drop('Credit_Score', axis=1)
    y_train = data_train['Credit_Score']
    X_test = data_test.drop('Credit_Score', axis=1)
    y_test = data_test['Credit_Score']

    numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = X_train.select_dtypes(exclude=['int64', 'float64']).columns

    scaler = StandardScaler()
    X_train_scaled = X_train.copy()
    X_test_scaled = X_test.copy()
    
    X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
    X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

    return X_train_scaled, X_test_scaled, y_train, y_test

In [33]:
"""
Función para crear un primer modelo con el método de Random Forest. 
Esta función entrará con los datos normalizados para poder proponer una predicción y una probabilidad asociada. 

EXPLICAR UN POCO MÁS QUE ES RF

"""

#RF FALTA DOCUMENTACION Y CAMBIAR COLUMNA
def random_forest(X_train_scaled,X_test_scaled, y_train):
    rf_model = RandomForestClassifier(random_state=100)
    rf_model.fit(X_train_scaled, y_train)
    rf_probs = rf_model.predict_proba(X_test_scaled)[:, 1]
    rf_preds = rf_model.predict(X_test_scaled)
    
    return rf_probs,rf_preds

In [34]:
def logistic_regression(X_train_scaled, y_train):
    lr_model = LogisticRegression(max_iter=1000, random_state=100)
    lr_model.fit(X_train_scaled, y_train)
    lr_probs = lr_model.predict_proba(X_test_scaled)[:, 1]
    lr_preds = lr_model.predict(X_test_scaled)
    
    return lr_probs,lr_preds

In [35]:
# Crear el modelo CatBoost
def catboost_model(X_train_scaled, y_train_scaled, cat_features=None, iterations=1000, learning_rate=0.03, depth=6):
    model = CatBoostClassifier(
        iterations=iterations,
        learning_rate=learning_rate,
        depth=depth,
        cat_features=cat_features,
        verbose=0 
    )
    
    return model.fit(X_train_scaled, y_train_scaled)

In [36]:
def xgboost_model(X_train_scaled, y_train_scaled, task_type='classification', **params):
   
    if task_type == 'classification':
        model = XGBClassifier(**params)
    else:
        model = XGBRegressor(**params)
    
    return model.fit(X_train_scaled, y_train_scaled)

In [37]:
def get_roc_metrics(y_test, probs):
    fpr, tpr, _ = roc_curve(y_test, probs)
    auc = roc_auc_score(y_test, probs)
    return fpr, tpr, auc

In [38]:
"""
Función para cargar la info del modelo
Separar en trian y test 
Función para entrenador el modelo con RF
Función para entrenador el modelo con Regresion 
Función para entrenador el modelo con ---
fUNCION PARA COMBINARLOS
Función para evaluar los modelos 
Fuincion Auroc
Funcion matriz
Función para que el usuario inserte el cliente a evaluar
¿Qué hacer en caso de que no se tenga un dato?
"""

'\nFunción para cargar la info del modelo\nSeparar en trian y test \nFunción para entrenador el modelo con RF\nFunción para entrenador el modelo con Regresion \nFunción para entrenador el modelo con ---\nfUNCION PARA COMBINARLOS\nFunción para evaluar los modelos \nFuincion Auroc\nFuncion matriz\nFunción para que el usuario inserte el cliente a evaluar\n¿Qué hacer en caso de que no se tenga un dato?\n'

In [39]:
"""
Función para cargar y leer los datos con la información crediticia que se utilizará para entrenar y validar el modelo.

El archivo que contenga estos datos debe de ser presentado en formato csv, de lo contrario, un mensaje de error hará saber
al usuario que el archivo no cumple con el formato solicitado.
El archivo debe de contener las siguientes 12 columnas en el orden que a continuación se presenta, cuidando
que estas correspondan a las primeras 12 columnas del documento: Interest_Rate, Delay_from_due_date,
Payment_of_Min_Amount, Outstanding_Debt, Num_Credit_Inquiries, Num_of_Delayed_Payment, Num_of_Loan, 
Credit_Utilization_Ratio, Num_Bank_Accounts, Num_Credit_Card, Credit_Mix, Credit_History_Age. 

Clasifiación para las variables que no son numéricas:
•Payment_of_Min_Amount: Yes, No, NM (Not Mentioned)
•Credit_Mix: Good, Standard, Poor

También es importante mencionar que el Interest_Rate se debe indicar como números enteros, no como porcentaje.
Para Outstanding_Debt, Credit_Utilization_Ratio y Credit_History_Age se pueden usar puntos decimales y para las 
restantes son números enteros.
El Delay_from_due_date debe indicarse en número de días.

En caso de tener valores faltantes se pondra cero para el caso de las columnas numéricas y NA para las categóricas. 
"""
def insert_credit_info(info):
    if not info.lower().endswith('.csv'):
        raise ValueError("El formato del archivo proporcionado no concuerda con el solicitado.")
    
    return pd.read_csv(info)

#Evaluar el modelo en lso datos porporcionado

In [40]:
data = "data_train_cleaned_final.csv"

In [42]:
data = get_data(data)

In [48]:
data_train, data_test = split_train_test(data,target_column="Credit_Score", random_state=100)

In [57]:
X_train_scaled,X_test_scaled, y_train, y_test= prepare_variables(data_train,data_test)

C:\Users\PAVILION\anaconda\lib\site-packages\sklearn\utils\extmath.py:985: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\PAVILION\anaconda\lib\site-packages\sklearn\utils\extmath.py:990: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\PAVILION\anaconda\lib\site-packages\sklearn\utils\extmath.py:1020: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count
